# References

- [python reference](https://docs.python.org/3/reference/datamodel.html)
  - Must read, if you can

In [1]:
Ellipsis is ...

True

## `__name__` v.s `__qualname__`

In [2]:
class A(object):
    class B(object):
        def m():
            pass

In [3]:
A.B.__qualname__

'A.B'

In [4]:
A.B.__name__

'B'

In [5]:
A.B.m.__qualname__, A.B.m.__name__

('A.B.m', 'm')

In [6]:
A.B.m.__module__

'__main__'

In [7]:
def foo(a = 1, b=[], *, c = 3):
    b.append(3)
    return a+1, b, c

In [8]:
foo.__defaults__

(1, [])

In [9]:
foo()

(2, [3], 3)

In [10]:
foo.__defaults__

(1, [3])

In [11]:
foo.__kwdefaults__ # kw only defaults

{'c': 3}

In [12]:
def bar():
    return c+1
bar()

NameError: name 'c' is not defined

In [13]:
c = 2

In [14]:
bar()

3

In [15]:
'c' in bar.__globals__

True

In [16]:
bar.__globals__ is globals()

True

In [17]:
bar.__dict__["c"] = 1

In [18]:
bar()

3

In [19]:
bar.__closure__ is None

True

In [20]:
def _():
    for a in range(10):
        def hey():
            return a+1
    return hey
hey = _()

In [21]:
hey.__closure__[0].cell_contents

9

## Magic Methods

- [python reference](https://docs.python.org/3/reference/datamodel.html#emulating-container-types)

In [22]:
class MyObject(object):
    __slots__ = ("_search_key", "_slice", "name")
    
    def __init__(self, name="myobj"):
        self.name = name
        self._search_key = {}
        self._slice = []
    
    # emulate function object
    def __call__(self, *args, **kwargs):
        """instance will work like function
        """
        print("{} is called".format(self.name))
        print(args)
        print(kwargs)
        
    # container
    def append(self, value):
        self._slice.append(value)
    
    def __len__(self):
        return len(self._search_key) + len(self)
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            print("slicing...")
            return self._do_slice(key)
        
        print("searching..")
        return self._do_search(key)
    
    def __setitem__(self, key, value):
        print("setting item!")
        self._search_key[key] = value
    
    def __delitem__(self, key):
        print("delete key!")
        if key in self._search_key:
            del self._search_key[key]
            
    def __iter__(self):
        print("iter!")
        return iter([1, 2, 3, 4, 5])
    
    def __reversed__(self):
        print("reverse!")
        return iter([5, 4, 3, 2, 1])
    
    def __contains__(self, value):
        print("contain nothing!!")
        print("value:", value)
        return False
    
    def _do_slice(self, a_slice):
        return self._slice[a_slice]
    
    def _do_search(self, key):
        ret = self._search_key.get(key, None)
        if ret is None:
            print("{} not found, so sad...".format(key))
        return ret

In [23]:
ob = MyObject()

In [24]:
ob() # due to __call__

myobj is called
()
{}


In [25]:
ob[1:2]

slicing...


[]

In [26]:
ob[1]

searching..
1 not found, so sad...


In [27]:
ob[1] = 3

setting item!


In [28]:
ob[1]

searching..


3

In [29]:
del ob[1]

delete key!


In [30]:
for i in ob:
    print(i)

iter!
1
2
3
4
5


In [31]:
for i in reversed(ob):
    print(i)

reverse!
5
4
3
2
1


In [32]:
3 in ob

contain nothing!!
value: 3


False

In [33]:
class MyDict(dict):
    
    def __getitem__(self, key):
        print("get item")
        return super(MyDict, self).__getitem__(key)
    
    def __missing__(self, key):
        print("missing key, so sad...")
        if not isinstance(key, str):
            raise TypeError("key should be of type str, get {}".format(type(key)))
        else:
            raise KeyError("key not found: {}".format(key))

In [34]:
d = MyDict()

In [35]:
d[1] = 2

In [36]:
d[3]

get item
missing key, so sad...


TypeError: key should be of type str, get <class 'int'>

In [37]:
d['a']

get item
missing key, so sad...


KeyError: 'key not found: a'

## Numeric Types

- [reference](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types)
- Basic rule: if `self.__xxx__(other)` return `NotImplemented`, python interpreter will call `other.__'r'xxx__(self)` instead
  - ex: `a+b` --> `a.__add__(b)` --> `NotImplemented` --> `b.__radd__(a)`
- operators: `self` will be the first operand, `other` will be the second one.
  - `+`: `__add__(self, other)`
  - `-`: `__sub__(self, other)`
  - `*`: `__mul__(self, other)`
  - `@`: `__matmul__(self, other)`
  - `/`: `__truediv__(self, other)`
  - `//`: `__floordiv__(self, other)`
  - `%`: `__mod__(self, other)`
  - `divmod`: `__divmod__(self, other)`, should be equivalent to `__floordiv__` and `__mod__`
    - that is, it will return (`self // other`, `self % other`)
  - `pow` or `**`: `__pow__(self, other)` or `__pow__(self, other, ternary=None)`, which's equivalent to `self ** other % ternary`
  - `<<`: `__lshift__(self, other)`, left shift
  - `>>`: `__rshift__(self, other)`, right shift
  - `&`: `__and__(self, other)`, bitwise and
  - `^`: `__xor__(self, other)`, exclusive or
  - `|`: `__or__(self, other)`, bitwise or

In [38]:
divmod(10, 3)

(3, 1)

In [39]:
# simple example
# for brevity, I only implement <<
class MagicObject(object):
    
    def __init__(self, value):
        self.value = int(value)
    
    def __lshift__(self, other):
        print("left shift in MagicObject!")
        return self.value << other.value
    
    def __rlshift__(self, other):
        print("rlshift in MagicObject!")
        return other.value << self.value

class MagicInt(object):
    
    def __init__(self, value):
        self.value = value
    
    def __lshift__(self, other):
        print("lshift in MagicInt! --> NotImplemented")
        return NotImplemented

In [40]:
i = MagicInt(3)
o = MagicObject(2)

In [41]:
i << o

lshift in MagicInt! --> NotImplemented
rlshift in MagicObject!


12

In [42]:
o << i

left shift in MagicObject!


16

- augmented arithmetic assignments
  - such as `+=`, `-=`, `<<=`, ...etc
- `__xxx__` <---> `__ixxx__`
- [good reading material](https://docs.python.org/3/faq/programming.html#faq-augmented-assignment-tuple-error)

In [43]:
# example implementation of augmented arithmetic assignment
class MyFloat(float):
    
    def __iadd__(self, other):
        print("iadd in MyFloat")
        result = self + other
        return MyFloat(result)
    
    def __str__(self):
        s = super(MyFloat, self).__str__()
        return "MyFloat({})".format(s)

In [44]:
f = MyFloat(3)

In [45]:
f += 1
print(f)

iadd in MyFloat
MyFloat(4.0)


In [46]:
class OOO(object):
    
    def __del__(self):
        print("delete!")
        s = super(OOO, self)
        if hasattr(s, '__del__'):
            s.__del__()
        else:
            print("parent has no delete hook!")

In [47]:
o = OOO()
del o

delete!
parent has no delete hook!


In [48]:
class CCC(OOO, object):
    def __del__(self):
        print("delete in CCC")
        s = super(CCC, self)
        if hasattr(s, "__del__"):
            print("[CCC] calling parent delete hook")
            s.__del__()
        else:
            print("[CCC] parent has no delete hook!")

In [49]:
c = CCC()
del c

delete in CCC
[CCC] calling parent delete hook
delete!
parent has no delete hook!
